In [7]:
import xarray as xr
import pandas as pd
import numpy as np
import dask.dataframe as dd
import matplotlib.pyplot as plt
from preprocessing import XarrayPreprocessing, PandasPreprocessing


In [8]:
hornsea_1_dwd_1 = xr.open_dataset("data/dwd_icon_eu_hornsea_1_20200920_20231027.nc")
hornsea_1_dwd_2 = xr.open_dataset("data/dwd_icon_eu_hornsea_1_20231027_20240108.nc")
hornsea_1_dwd_3 = xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240108_20240129.nc")
hornsea_1_dwd_4 = xr.open_dataset("data/dwd_icon_eu_hornsea_1_20240129_20240519.nc")

hornsea_1_ncep_1 = xr.open_dataset("data/ncep_gfs_hornsea_1_20200920_20231027.nc")
hornsea_1_ncep_2 = xr.open_dataset("data/ncep_gfs_hornsea_1_20231027_20240108.nc")
hornsea_1_ncep_3 = xr.open_dataset("data/ncep_gfs_hornsea_1_20240108_20240129.nc")
hornsea_1_ncep_4 = xr.open_dataset("data/ncep_gfs_hornsea_1_20240129_20240519.nc")

In [9]:
hornsea_1 = [hornsea_1_dwd_1, hornsea_1_dwd_2, hornsea_1_dwd_3, hornsea_1_dwd_4, hornsea_1_ncep_1, hornsea_1_ncep_2, hornsea_1_ncep_3, hornsea_1_ncep_4]

for dataset in hornsea_1:
    print(dataset.dims)

FrozenMappingWarningOnValuesAccess({'ref_datetime': 4467, 'valid_datetime': 93, 'latitude': 6, 'longitude': 6})
FrozenMappingWarningOnValuesAccess({'valid_time': 93, 'reference_time': 293, 'latitude': 6, 'longitude': 6})
FrozenMappingWarningOnValuesAccess({'reference_time': 85, 'valid_time': 93, 'latitude': 6, 'longitude': 6})
FrozenMappingWarningOnValuesAccess({'reference_time': 441, 'valid_time': 93, 'latitude': 6, 'longitude': 6})
FrozenMappingWarningOnValuesAccess({'ref_datetime': 4526, 'valid_datetime': 209, 'latitude': 3, 'longitude': 3})
FrozenMappingWarningOnValuesAccess({'reference_time': 293, 'valid_time': 209, 'latitude': 3, 'longitude': 3})
FrozenMappingWarningOnValuesAccess({'reference_time': 84, 'valid_time': 209, 'latitude': 3, 'longitude': 3})
FrozenMappingWarningOnValuesAccess({'reference_time': 416, 'valid_time': 209, 'latitude': 3, 'longitude': 3})


In [10]:
for dataset in ["hornsea_1_dwd_1", "hornsea_1_ncep_1"]:
    globals()[dataset] = globals()[dataset].rename({"ref_datetime":"ref_time", "valid_datetime":"val_time", "latitude":"lat", "longitude":"long"})

for dataset in ["hornsea_1_dwd_2", "hornsea_1_dwd_3", "hornsea_1_dwd_4", "hornsea_1_ncep_2", "hornsea_1_ncep_3", "hornsea_1_ncep_4"]:
    globals()[dataset] = globals()[dataset].rename({"reference_time":"ref_time", "valid_time":"val_time", "latitude":"lat", "longitude":"long"})

hornsea_1 = [hornsea_1_dwd_1, hornsea_1_dwd_2, hornsea_1_dwd_3, hornsea_1_dwd_4, hornsea_1_ncep_1, hornsea_1_ncep_2, hornsea_1_ncep_3, hornsea_1_ncep_4]

In [11]:
hornsea_1_dwd = xr.concat(hornsea_1[:4], "ref_time")
hornsea_1_ncep = xr.concat(hornsea_1[4:], "ref_time")

In [12]:
hornsea_1_dwd_preprocessor = XarrayPreprocessing(hornsea_1_dwd)
hornsea_1_ncep_preprocessor = XarrayPreprocessing(hornsea_1_ncep)

for object in (hornsea_1_dwd_preprocessor, hornsea_1_ncep_preprocessor):
    (object
     .remove_duplicates(dim="ref_time")
     .remove_nans(dim="ref_time", how="all")
     .fill_nans(value={"RelativeHumidity": object.dataset["RelativeHumidity"].mean(dim=["ref_time", "val_time"]),
                     "Temperature": object.dataset["Temperature"].mean(dim=["ref_time", "val_time"]),
                     "WindDirection": object.dataset["WindDirection"].mean(dim=["ref_time", "val_time"]),
                     "WindDirection:100": object.dataset["WindDirection:100"].mean(dim=["ref_time", "val_time"]),
                     "WindSpeed": object.dataset["WindSpeed"].mean(dim=["ref_time", "val_time"]),
                     "WindSpeed:100": object.dataset["WindSpeed:100"].mean(dim=["ref_time", "val_time"])})
     .remove_outliers(variables = list(object.dataset.data_vars), dim=["ref_time", "val_time"])
     .convert_val_time("ref_time", "val_time")
    )
hornsea_1_dwd_cleaned = hornsea_1_dwd_preprocessor.get_dataset()
hornsea_1_ncep_cleaned = hornsea_1_ncep_preprocessor.get_dataset()

df_hornsea_1_dwd = hornsea_1_dwd_cleaned.to_dataframe().reset_index()
df_hornsea_1_ncep = hornsea_1_ncep_cleaned.to_dataframe().reset_index()

In [13]:
def merge_by_chunks(df_1, df_2, chunk_size, how="inner", on=None, left_on=None, right_on=None):
    """Merges two dataframes by a given chunksize."""
    df_2_list = [df_2[i:i+chunk_size] for i in range(0, df_2.shape[0], chunk_size)]
    df_merged = pd.DataFrame()
    
    for df in df_2_list:
        df_merged  = pd.concat([df_merged, df_1.merge(df, how=how, on=on, left_on=left_on, right_on=right_on)])

    return df_merged

In [14]:
df_energy_2 = pd.read_csv("data\Energy_Data_20200920_20240118.csv")
df_energy_3 = pd.read_csv("data\Energy_Data_20240119_20240519.csv")
df_energy = pd.concat([df_energy_2, df_energy_3])

df_energy["dtm"] = pd.to_datetime(df_energy["dtm"])
df_energy["wind_mwh_credit"] = 0.5 * df_energy["Wind_MW"] - df_energy["boa_MWh"]
df_energy["solar_mwh_credit"] = 0.5 * df_energy["Solar_MW"] - df_energy["boa_MWh"]

In [15]:
energy_preprocessor = PandasPreprocessing(df_energy)

nan_columns = [index for index in energy_preprocessor.get_dataframe().isna().sum().index if energy_preprocessor.get_dataframe().isna().sum()[index] > 0]
energy_preprocessor.dataframe["week"] = energy_preprocessor.dataframe["dtm"].dt.isocalendar().week

df_energy = (energy_preprocessor
             .remove_nans(how="all", subset=nan_columns)
             .fill_nans(df_energy, nan_columns, mean_over="week")
             .remove_outliers(columns=df_energy.columns, mean_over="week")
             .get_dataframe())

In [16]:
df_hornsea_1 = pd.concat([df_hornsea_1_dwd, df_hornsea_1_ncep]).reset_index()
df_hornsea_1 = pd.merge(df_hornsea_1, df_energy, how="inner", left_on="val_datetime", right_on="dtm")

In [17]:
df_hornsea_1

,index,lat,long,val_datetime,RelativeHumidity,Temperature,WindDirection,WindDirection:100,WindSpeed,WindSpeed:100,...,Solar_MW,Solar_capacity_mwp,Solar_installedcapacity_mwp,Wind_MW,SS_Price,boa_MWh,DA_Price,wind_mwh_credit,solar_mwh_credit,week
0,0,53.77,1.702,2020-09-20 00:00:00+00:00,86.991142,15.276886,59.552521,60.032074,10.064903,11.884751,...,0.000000,2130.537493,2228.208777,996.284,2.50000,0.0,32.17,498.142,0.000000,38
1,1,53.77,1.702,2020-09-20 01:00:00+00:00,85.712303,15.340668,59.803680,60.287476,9.756837,11.560445,...,0.000000,2130.537493,2228.208777,941.044,47.95000,0.0,32.00,470.522,0.000000,38
2,2,53.77,1.702,2020-09-20 02:00:00+00:00,84.477051,15.507812,60.068024,60.598938,9.450724,11.330547,...,0.000000,2130.537493,2228.208777,918.432,28.95000,0.0,31.99,459.216,0.000000,38
3,3,53.77,1.702,2020-09-20 03:00:00+00:00,84.295494,15.484406,57.425568,58.001556,9.598772,11.505955,...,0.000000,2130.537493,2228.208777,978.682,30.90000,0.0,32.54,489.341,0.000000,38
4,4,53.77,1.702,2020-09-20 04:00:00+00:00,83.777802,15.506439,56.003479,56.508301,9.673996,11.625600,...,0.000000,2130.537493,2228.208777,1002.902,29.00000,0.0,32.28,501.451,0.000000,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27633766,9999206,54.09,2.022,2024-05-19 19:00:00+00:00,89.900002,11.655304,14.693939,19.329437,6.533110,8.871146,...,36.898585,2573.542092,2741.831037,227.554,56.01000,0.0,87.04,113.777,18.449292,20
27633767,9999207,54.09,2.022,2024-05-19 20:00:00+00:00,92.199997,11.350006,16.174347,20.735199,6.740209,9.107319,...,0.000000,2573.542092,2741.831037,226.080,56.11000,0.0,69.59,113.040,0.000000,20
27633768,9999208,54.09,2.022,2024-05-19 21:00:00+00:00,95.300003,11.114685,17.880341,21.800812,6.763072,9.033005,...,0.000000,2573.542092,2741.831037,231.590,106.39986,0.0,76.81,115.795,0.000000,20
27633769,9999209,54.09,2.022,2024-05-19 22:00:00+00:00,95.900002,10.902100,19.426208,21.592560,7.021616,8.815755,...,0.000000,2573.542092,2741.831037,197.090,102.00023,0.0,71.11,98.545,0.000000,20


In [18]:
# TODO: Explo vor und nach Preprocessing (vor und nach outliers, evt nans)